# This notebook melts the lfc, q-value dataframes in a format that is used by the Dash website. 

Could use some cleaning up. 

In [1]:
import pandas as pd
import numpy as np
import pathlib
import glob
import os

## Check if trackviews are complete

In [2]:
col_desc = pd.read_excel('../../data/column_descriptors_standardized_021023.xlsx', engine='openpyxl')

FileNotFoundError: [Errno 2] No such file or directory: '../../data/column_descriptors_standardized_021023.xlsx'

In [ ]:
col_desc.head(10)

In [ ]:
col_desc.column_ID_SI.unique()

In [ ]:
# check if trackviews exist
# trackview_folders = list(pathlib.Path(
#     r'C:\Users\light\Dropbox\tuberculosis\tuberculosis\TrackView').glob('*'))
# trackview_names = [os.path.basename(x) for x in trackview_folders]

In [ ]:
# all_conditions = list(col_desc['control']) + list(col_desc['experimental'])

In [ ]:
# set(all_conditions).difference(set(trackview_names))

## make_std_data_dash

In [ ]:
col_desc = pd.read_excel('../../data/column_descriptors_standardized_021023.xlsx', engine='openpyxl')
col_desc = col_desc.rename(columns={'column_ID_2': 'column_ID_std'})

In [ ]:
lfc = pd.read_csv(
    '../../data/standardized_data/result_logfc_matrix_2023_02_20_CC_BxD_processed.csv')

In [ ]:
qval = pd.read_csv(
    '../../data/standardized_data/result_qval_matrix_2023_02_20_CC_BxD_processed.csv')

In [ ]:
qval

In [ ]:
gene_names = pd.read_excel(
    '../../data/annotations/DeJesus_mbio.xlsx', header=1, engine='openpyxl')
gene_names.head()

In [ ]:
gene_names = gene_names.rename(
    columns={'Name': 'gene_name', 'ORF ID': 'Rv_ID'})

In [ ]:
set(lfc.columns).difference(set(qval.columns))

In [ ]:
set(qval.columns).difference(set(lfc.columns))

In [ ]:
col_ID_standardized = list(col_desc.column_ID_std.unique())

In [ ]:
set(qval.columns).difference(set(col_ID_standardized))

In [ ]:
set(col_ID_standardized).difference(set(qval.columns))

In [ ]:
lfc.head()

In [ ]:
lfc = lfc.merge(gene_names[['Rv_ID', 'gene_name', 'Description']], how='left')

In [ ]:
lfc = lfc.melt(id_vars=['Rv_ID', 'gene_name', 'Description'],
               var_name='Expt', value_name='log2FC')
qval = qval.melt(id_vars=['Rv_ID'], var_name='Expt', value_name='q-val')

In [ ]:
print(lfc.shape, qval.shape)

In [ ]:
lfc.head()

In [ ]:
std_data = lfc.merge(qval, how='left', on=['Rv_ID', 'Expt'])

In [ ]:
std_data.head()

In [ ]:
std_data.isna().sum()

In [ ]:
std_data['gene_name'] = std_data.apply(
    lambda x: '-' if pd.isna(x.gene_name) else x.gene_name, axis=1)

In [ ]:
std_data

In [ ]:
std_data.isna().sum()

In [ ]:
std_data[std_data['log2FC'].isna()]

In [ ]:
std_data = std_data[~std_data['log2FC'].isna()]

In [ ]:
std_data.isna().sum()

In [ ]:
def discretize_q_values(q_val):
    if pd.isna(q_val):
        return np.nan
    if q_val < 0.01:
        q_val_d = 3
    elif q_val < 0.05:
        q_val_d = 2
    else:
        q_val_d = 1
    return q_val_d

In [ ]:
std_data['q_val_D'] = std_data['q-val'].apply(discretize_q_values)

In [ ]:
std_data.head()

In [ ]:
df_uk = pd.read_csv(
    '../../data/annotations/uniprot_annotation_scores.csv')

In [ ]:
df_uk.head()

In [ ]:
std_data = std_data.merge(
    df_uk, how='left', on='Rv_ID').rename(columns={'Annotation': 'annotation_score'})

In [ ]:
std_data.isna().sum()

In [ ]:
# have to save as tsv because descriptions have commas
std_data.to_csv('../../data/standardized_data_dash.tsv', index=False, sep='\t')

## Make SI data

In [ ]:
%reset -f

In [ ]:
import pandas as pd
import numpy as np
import pathlib
import glob
import os

In [ ]:
col_desc = pd.read_excel('../../data/column_descriptors_standardized_021023.xlsx', engine='openpyxl')

In [ ]:
col_desc.head(10)

In [ ]:
col_desc = col_desc.rename(columns={'column_ID_2': 'column_ID_std'})

In [ ]:
lfc_si = pd.read_csv(
    '../../data/SI_datasets/SI_log2FC.csv')
lfc_si = lfc_si.drop(columns='gene_name')

In [ ]:
qval_si = pd.read_csv(
    '../../data/SI_datasets/SI_qval.csv')
qval_si = qval_si.drop(columns='gene_name')

In [ ]:
print(lfc_si.shape, qval_si.shape)

In [ ]:
lfc_si

In [ ]:
gene_names = pd.read_excel(
    '../../data/annotations/DeJesus_mbio.xlsx', header=1, engine='openpyxl')
gene_names.head()

In [ ]:
gene_names = gene_names.rename(
    columns={'Name': 'gene_name', 'ORF ID': 'Rv_ID'})

In [ ]:
set(lfc_si.columns).difference(set(qval_si.columns))

In [ ]:
set(qval_si.columns).difference(set(lfc_si.columns))

In [ ]:
col_ID_SI = list(col_desc.column_ID_SI.unique())

In [ ]:
set(qval_si.columns).difference(set(col_ID_SI))

In [ ]:
# fix this in metadata!
set(col_ID_SI).difference(set(qval_si.columns))

In [ ]:
lfc_si.head()

In [ ]:
lfc_si = lfc_si.merge(
    gene_names[['Rv_ID', 'gene_name', 'Description']], how='left')

In [ ]:
lfc_si.shape

In [ ]:
lfc_si = lfc_si.melt(id_vars=['Rv_ID', 'gene_name', 'Description'],
                     var_name='Expt', value_name='log2FC')
qval_si = qval_si.melt(id_vars=['Rv_ID'], var_name='Expt', value_name='q-val')

In [ ]:
print(lfc_si.shape, qval_si.shape)

In [ ]:
lfc_si.head()

In [ ]:
si_data = lfc_si.merge(qval_si, how='left', on=['Rv_ID', 'Expt'])

In [ ]:
si_data.head()

In [ ]:
si_data.isna().sum()

In [ ]:
si_data['gene_name'] = si_data.apply(
    lambda x: '-' if pd.isna(x.gene_name) else x.gene_name, axis=1)

In [ ]:
si_data

In [ ]:
si_data.dtypes
#si_data['q-val'] = si_data['q-val'].replace({'no replicates':np.nan})

In [ ]:
si_data.isna().sum()

In [ ]:
def discretize_q_values(q_val):
    if pd.isna(q_val):
        return np.nan
    if q_val < 0.01:
        q_val_d = 3
    elif q_val < 0.05:
        q_val_d = 2
    else:
        q_val_d = 1
    return q_val_d

In [ ]:
si_data['q_val_D'] = si_data['q-val'].apply(discretize_q_values)

In [ ]:
si_data.head()

In [ ]:
df_uk = pd.read_csv(
    '../../data/annotations/uniprot_annotation_scores.csv')
df_uk.head()

In [ ]:
si_data = si_data.merge(df_uk, how='left', on='Rv_ID').rename(columns={'Annotation': 'annotation_score'})

In [ ]:
si_data.isna().sum()

In [ ]:
si_data.to_csv('../../data/si_data_dash.tsv', index=False, sep='\t')

## Make expt metadata

In [ ]:
%reset -f

In [ ]:
si_missing = {'129s1.SvImJ_vs_in_vitro',
 'A.J_vs_in_vitro',
 'BXD102_vs_in_vitro_H37Rv',
 'BXD29_vs_in_vitro_H37Rv',
 'BXD39_vs_in_vitro_H37Rv',
 'BXD40_vs_in_vitro_H37Rv',
 'BXD48a_vs_in_vitro_H37Rv',
 'BXD51_vs_in_vitro_H37Rv',
 'BXD54_vs_in_vitro_H37Rv',
 'BXD56_vs_in_vitro_H37Rv',
 'BXD60_vs_in_vitro_H37Rv',
 'BXD62_vs_in_vitro_H37Rv',
 'BXD67_vs_in_vitro_H37Rv',
 'BXD69_vs_in_vitro_H37Rv',
 'BXD73_vs_in_vitro_H37Rv',
 'BXD73b_vs_in_vitro_H37Rv',
 'BXD77_vs_in_vitro_H37Rv',
 'BXD79_vs_in_vitro_H37Rv',
 'BXD90_vs_in_vitro_H37Rv',
 'BXD93_vs_in_vitro_H37Rv',
 'BXD9_vs_in_vitro_H37Rv',
 'C57BL.6J_vs_in_vitro',
 'C57BL.6J_vs_in_vitro_H37Rv',
 'CC001.Unc_vs_in_vitro',
 'CC002.Unc_vs_in_vitro',
 'CC003.Unc_vs_in_vitro',
 'CC004.TauUnc_vs_in_vitro',
 'CC005.TauUnc_vs_in_vitro',
 'CC006.TauUnc_vs_in_vitro',
 'CC007.Unc_vs_in_vitro',
 'CC008.GeniUnc_vs_in_vitro',
 'CC009.Unc_vs_in_vitro',
 'CC010.GeniUnc_vs_in_vitro',
 'CC011.Unc_vs_in_vitro',
 'CC012.GeniUnc_vs_in_vitro',
 'CC013.GeniUnc_vs_in_vitro',
 'CC015.Unc_vs_in_vitro',
 'CC016.GeniUnc_vs_in_vitro',
 'CC017.Unc_vs_in_vitro',
 'CC018.Unc_vs_in_vitro',
 'CC019.TauUnc_vs_in_vitro',
 'CC021.Unc_vs_in_vitro',
 'CC022.GeniUnc_vs_in_vitro',
 'CC023.GeniUnc_vs_in_vitro',
 'CC024.GeniUnc_vs_in_vitro',
 'CC025.GeniUnc_vs_in_vitro',
 'CC027.GeniUnc_vs_in_vitro',
 'CC028.GeniUnc_vs_in_vitro',
 'CC030.GeniUnc_vs_in_vitro',
 'CC031.GeniUnc_vs_in_vitro',
 'CC032.GeniUnc_vs_in_vitro',
 'CC033.GeniUnc_vs_in_vitro',
 'CC034.Unc_vs_in_vitro',
 'CC035.Unc_vs_in_vitro',
 'CC037.TauUnc_vs_in_vitro',
 'CC038.GeniUnc_vs_in_vitro',
 'CC039.Unc_vs_in_vitro',
 'CC040.TauUnc_vs_in_vitro',
 'CC041.TauUnc_vs_in_vitro',
 'CC042.GeniUnc_vs_in_vitro',
 'CC043.GeniUnc_vs_in_vitro',
 'CC044.Unc_vs_in_vitro',
 'CC045.GeniUnc_vs_in_vitro',
 'CC046.Unc_vs_in_vitro',
 'CC047.Unc_vs_in_vitro',
 'CC051.TauUnc_vs_in_vitro',
 'CC056.GeniUnc_vs_in_vitro',
 'CC057.Unc_vs_in_vitro',
 'CC059.TauUnc_vs_in_vitro',
 'CC061.GeniUnc_vs_in_vitro',
 'CC065.Unc_vs_in_vitro',
 'Cast.EiJ_vs_in_vitro',
 'DBA.2J_vs_in_vitro_H37Rv',
 'IFNG_vs_in_vitro',
 'NOD.ShiLtJ_vs_in_vitro',
 'NOS_vs_in_vitro',
 'NZO.H1LtJ_vs_in_vitro',
 'PWK.PhJ_vs_in_vitro',
 'Phox_vs_in_vitro',
 'Rag_vs_in_vitro',
 'Wsb.Eij_vs_in_vitro',
 'mbio_H37Rv_vs_in_vitro'}

In [ ]:
import pandas as pd
import numpy as np
import pathlib
import glob
import os

In [ ]:
col_desc = pd.read_excel('../../data/column_descriptors_standardized_021023.xlsx', engine='openpyxl')

In [ ]:
col_desc = col_desc.rename(columns={'column_ID_2': 'column_ID_std'})
col_desc = col_desc.drop(columns=['column_ID', 'wig_files'])
col_desc = col_desc.drop(columns=[c for c in col_desc.columns if 'Unnamed' in c])

In [ ]:
col_desc.loc[col_desc['column_ID_SI'].isin(si_missing), 'column_ID_SI'] = np.nan

In [ ]:
col_desc.head()

In [ ]:
col_desc.isna().sum()

In [ ]:
col_desc.drop(columns=['year', 'journal',
                       'first_author', 'last_author', 'control', 'experimental', 'stat_analysis'], inplace=True)

In [ ]:
fill_na_cols = [c for c in col_desc.columns if c not in [
    'control', 'experimental', 'column_ID_std', 'column_ID_SI', 'plot_SI_graph']]

In [ ]:
col_desc[fill_na_cols] = col_desc[fill_na_cols].fillna('')

In [ ]:
col_desc['num_replicates_control'] = col_desc['num_replicates_control'].apply(lambda x: int(x) if x!='' else '')
col_desc['num_replicates_experimental'] = col_desc['num_replicates_experimental'].apply(lambda x: int(x) if x!='' else '')

In [ ]:
rename_dict = {
    'num_replicates_control': 'num replicates control',
    'num_replicates_experimental': 'num replicates experimental',
    'in_vitro_cell_vivo': 'in vitro/cell/in vivo',
    'carbon_source': 'carbon source',
    'in_vitro_media': 'in vitro media',
    'stress_description': 'stress',
    'MicArr_or_TnSeq' : 'Microarray or TnSeq',
    'mouse_strain': 'mouse strain',
    'cell_type': 'cell type',
    'Mtb_strain': 'Mtb strain'
}
col_desc = col_desc.rename(columns = rename_dict)

In [ ]:
col_desc['stress'] = col_desc['stress'].apply(lambda x: x.replace('_', ' '))

In [ ]:
col_desc.to_csv('../../data/col_desc_dash.tsv', index=False, sep='\t')

## Make gene metadata

In [ ]:
%reset -f

In [ ]:
import pandas as pd
import numpy as np
import pathlib
import glob
import os

In [ ]:
si_data = pd.read_csv('../../data/si_data_dash.tsv', sep='\t')
si_data.head()

In [ ]:
std_data = pd.read_csv('../../data/standardized_data_dash.tsv', sep='\t')
std_data.head()

In [ ]:
#are all Rvs in si data there in std data?
si_Rvs = set(si_data.Rv_ID.to_list())
std_Rvs = set(std_data.Rv_ID.to_list())


In [ ]:
len(si_Rvs)

In [ ]:
len(std_Rvs)

In [ ]:
si_Rvs.difference(std_Rvs)

In [ ]:
si_Rvs.issubset(std_Rvs)

In [ ]:
# Therefore use std to get gene metadata

In [ ]:
gene_metadata = std_data[['Rv_ID', 'gene_name', 'Description']].drop_duplicates()

In [ ]:
gene_names = pd.read_excel(
    '../../data/annotations/DeJesus_mbio.xlsx', header=1, engine='openpyxl')
gene_names = gene_names.rename(
    columns={'ORF ID': 'Rv_ID'})
gene_names.head()

In [ ]:
gene_metadata = gene_metadata.merge(gene_names[['Rv_ID', 'Final Call']], how='left', on='Rv_ID')

In [ ]:
gene_metadata.isna().sum()

In [ ]:
gene_metadata['Description'] = gene_metadata['Description'].fillna('-')
gene_metadata['Final Call'] = gene_metadata['Final Call'].fillna('Missing')

In [ ]:
gene_metadata['Final Call'].value_counts()

In [ ]:
gene_metadata_rename = {
    'NE': 'Non essential',
    'ES': 'Essential',
    'GA': 'Growth advantage',
    'GD': 'Growth defect',
    'ESD': 'Essential domain'
}
gene_metadata['Final Call'] = gene_metadata['Final Call'].replace(gene_metadata_rename)

In [ ]:
gene_metadata

In [ ]:
func_cats = pd.read_excel('../../data/annotations/H37Rv_tuberculist_functional_categories.xlsx', engine='openpyxl')
func_cats.head()

In [ ]:
gene_metadata = gene_metadata.merge(func_cats[['Rv_ID', 'category']], on='Rv_ID', how='left').rename(columns = {'category': 'tuberculist_category'})

In [ ]:
gene_metadata['tuberculist_category'] = gene_metadata['tuberculist_category'].fillna('unknown')

In [ ]:
gene_metadata['tuberculist_category'].value_counts()

In [ ]:
umap_data = pd.read_csv('../../data/NN_for_dash.csv')
umap_data.head()

In [ ]:
umap_data_cols = [col for col in umap_data.columns if col!= 'Rv_ID']
umap_data['NN_joined'] = umap_data.apply(lambda x: ', '.join(x[umap_data_cols].values.tolist()), axis=1)

In [ ]:
umap_data.head()

In [ ]:
gene_metadata = gene_metadata.merge(umap_data[['Rv_ID', 'NN_joined']], on='Rv_ID', how='left')

In [ ]:
gene_metadata.isna().sum()

In [ ]:
gene_metadata.to_csv('../../data/gene_metadata_dash.tsv', index=False, sep='\t')